In [ ]:
import numpy as np
import sys
import os

from glob import glob

sys.path.append("..")
sys.path.append("../..")

from benchmark_helpers import NCFLOW_HYPERPARAMS, PATH_FORM_HYPERPARAMS
from lib.algorithms import NcfEpi, Objective, PathFormulation, POP
from lib.problem import Problem
from lib.graph_utils import check_feasibility

In [ ]:
num_paths, edge_disjoint, dist_metric = PATH_FORM_HYPERPARAMS

num_subproblems = 4
split_method = "random"
split_fraction = 0.75
algo_cls = PathFormulation

obj_str = "total_flow"

In [ ]:
def run_pf(problem):
    pf = PathFormulation(
        objective=Objective.get_obj_from_str(obj_str),
        num_paths=num_paths,
        edge_disjoint=edge_disjoint,
        dist_metric=dist_metric,
        DEBUG=False,
    )
    pf.solve(problem)
    return pf

In [ ]:
def run_pop(problem):
    pop = POP(
        objective=Objective.get_obj_from_str(obj_str),
        num_subproblems=num_subproblems,
        split_method=split_method,
        split_fraction=split_fraction,
        algo_cls=algo_cls,
        num_paths=num_paths,
        edge_disjoint=edge_disjoint,
        dist_metric=dist_metric,
        DEBUG=False,
    )
    pop.solve(problem)
    return pop

In [ ]:
def run_ncflow(problem):
    (
        num_paths,
        edge_disjoint,
        dist_metric,
        partition_cls,
        num_parts_scale_factor,
    ) = NCFLOW_HYPERPARAMS[problem.name]
    num_partitions_to_set = num_parts_scale_factor * int(np.sqrt(len(problem.G.nodes)))
    partitioner = partition_cls(num_partitions_to_set)

    ncflow = NcfEpi(
        objective=Objective.get_obj_from_str(obj_str),
        num_paths=num_paths,
        edge_disjoint=edge_disjoint,
        dist_metric=dist_metric,
        DEBUG=True,
        VERBOSE=True,
    )
    ncflow.solve(problem, partitioner)
    return ncflow

In [ ]:
def run_algos(problem, algo_labels_and_fns):
    print("Total demand: ", problem.total_demand)
    algos = {}
    results = []
    for (algo_label, run_algo_fn) in algo_labels_and_fns:
        algo = run_algo_fn(problem)
        check_feasibility(problem, [algo.sol_dict])
        algos[algo_label] = algo
        results.append("{}: {}".format(algo_label, algo.obj_val))
    print(", ".join(results))
    return algos

In [ ]:
def get_path_lengths_per_commod(problem, paths_dict):
    l = []
    for (_, (s_k, t_k, _)) in problem.commodity_list:
        s_k_t_k_paths = paths_dict[(s_k, t_k)]
        num_hops_per_path = [len(p) for p in s_k_t_k_paths]
        s_k_t_k_shortest_path, s_k_t_k_longest_path = min(num_hops_per_path), max(num_hops_per_path)
        l.append((s_k, t_k, s_k_t_k_shortest_path, s_k_t_k_longest_path))    
    return sorted(l, key=lambda x: x[-2] + x[-1], reverse=True)

In [ ]:
def generate_infinite_demands_problem(topo_name, commods_sorted_by_hops, distant=True):
    G = Problem._read_graph_graphml(os.path.join('../../topologies/topology-zoo', topo_name))
    capacity = list(G.edges.data('capacity'))[0][-1]
    num_nodes = len(G.nodes)
    tm = np.zeros((num_nodes, num_nodes))
    if distant:
        half_of_commods = commods_sorted_by_hops[:len(commods_sorted_by_hops)//2]
    else:
        half_of_commods = commods_sorted_by_hops[len(commods_sorted_by_hops)//2:]
    for (s, t, _, _) in half_of_commods:
        tm[s, t] = capacity

    problem = Problem.fixed_traffic_matrix_problem(G, traffic_matrix=tm)
    problem.name = topo_name
    return problem


In [ ]:
def get_dummy_prob_and_paths(topo_name):
    tm_fname_glob_pattern = '../../traffic-matrices/uniform/{}_uniform_*_1.0_*_traffic-matrix.pkl'.format(
        topo_name
    )
    tm_fname = list(glob(tm_fname_glob_pattern))[0]
    print(tm_fname)
    problem = Problem.from_file(os.path.join('../../topologies/topology-zoo', topo_name), tm_fname)
    paths_dict = PathFormulation.read_paths_from_disk_or_compute(problem,
                                                                   num_paths=num_paths,
                                                                   edge_disjoint=edge_disjoint,
                                                                   dist_metric=dist_metric)
    return problem, paths_dict

In [ ]:
def run_experiment(topo_name):
    dummy_prob, paths_dict = get_dummy_prob_and_paths(topo_name)
    commods_sorted_by_hops = get_path_lengths_per_commod(dummy_prob, paths_dict)
    infinite_demands_distant_pairs = generate_infinite_demands_problem(topo_name,
                                                                       commods_sorted_by_hops,
                                                                       distant=True)
    infinite_demands_close_pairs = generate_infinite_demands_problem(topo_name,
                                                                     commods_sorted_by_hops,
                                                                     distant=False)

    labels_and_algos_to_run = [("PF", run_pf), ("NCFlow", run_ncflow), ("POP", run_pop)]
    distant_algos = run_algos(infinite_demands_distant_pairs, labels_and_algos_to_run)
    close_algos = run_aglos(infinite_demands_close_pairs, labels_and_algos_to_run)
    
    pf_close = close_algos["PF"]
    ncflow_close = close_algos["NCFlow"]
    pop_close = close_algos["POP"]
    
    print("Close: NCFlow vs PF: {}, POP vs PF: {}".format(
        ncflow_close.obj_val / pf_close.obj_val,
        pop_close.obj_val / pf_close.obj_val
        )
    )
    
        
    pf_distant = distant_algos["PF"]
    ncflow_distant = distant_algos["NCFlow"]
    pop_distant = distant_algos["POP"]
    
    print("Distant: NCFlow vs PF: {}, POP vs PF: {}".format(
        ncflow_distant.obj_val / pf_distant.obj_val, pop_distant.obj_val / pf_distant.obj_val
        )
    )
    return pf_distant, pop_distant, ncflow_distant, pf_close, pop_close, ncflow_close


In [ ]:
run_experiment("Cogentco.graphml")

In [ ]:
run_experiment("Colt.graphml")

In [ ]:
run_experiment("Deltacom.graphml")

In [ ]:
run_experiment("DialtelecomCz.graphml")

In [ ]:
run_experiment("GtsCe.graphml")

In [ ]:
run_experiment("TataNld.graphml")

In [ ]:
run_experiment("UsCarrier.graphml")